In [44]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random

In [18]:
all_a = []
limit = 30
for i in tqdm(range(0, 100000, limit)):
    if i == 0:
        i = ''
    else:
        i = f'/+{i}'
    url = f'https://forum.lowyat.net/Kopitiam{i}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href') and a_.get('href').startswith('/topic')]
    a = list(set([a_ for a_ in a if len(a_.split('/')) == 3]))
    if not len(a):
        break
    all_a.extend(a)

 16%|███████████████                                                                              | 541/3334 [07:23<38:11,  1.22it/s]


In [21]:
all_a = list(set(all_a))

In [22]:
len(all_a)

16214

In [24]:
with open('Kopitiam-topics.json', 'w') as fopen:
    json.dump(all_a, fopen)

In [61]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_text(url):
    texts = []
    try:
        r_ = requests.get(url)
        soup = BeautifulSoup(r_.content, "lxml")
        divs = soup.find_all('div', {'class': 'post_text'})
        for div in divs:
            t = BeautifulSoup(str(div).replace('<br/>', '\n')).text.strip()
            if len(t):
                texts.append(t)
    except Exception as e:
        pass
    return texts

In [34]:
topic[1:].replace('/', '-')

'topic-2872926'

In [49]:
base_dir

'topic-2872926'

In [56]:
last_texts = []
last_texts = [1][:]
last_texts

[1]

In [63]:
!ls done

topic-2872926.json  topic-5357388.json	topic-5362286.json  topic-5367086.json
topic-2948341.json  topic-5357389.json	topic-5362287.json  topic-5367087.json
topic-3109672.json  topic-5357390.json	topic-5362288.json  topic-5367089.json
topic-3737709.json  topic-5357391.json	topic-5362290.json  topic-5367090.json
topic-3915561.json  topic-5357392.json	topic-5362292.json  topic-5367091.json
topic-3962229.json  topic-5357393.json	topic-5362293.json  topic-5367092.json
topic-4076777.json  topic-5357394.json	topic-5362297.json  topic-5367093.json
topic-4156178.json  topic-5357395.json	topic-5362300.json  topic-5367094.json
topic-4179386.json  topic-5357396.json	topic-5362302.json  topic-5367098.json
topic-4181884.json  topic-5357397.json	topic-5362303.json  topic-5367105.json
topic-4282023.json  topic-5357398.json	topic-5362304.json  topic-5367107.json
topic-4287114.json  topic-5357399.json	topic-5362305.json  topic-5367109.json
topic-4293892.json  topic-5357401.json	topic-5362307.json  topic

In [72]:
limit = 20
worker_size = 1
per_batch = worker_size * limit

for topic in tqdm(sorted(all_a)):
    
    base_dir = topic[1:].replace('/', '-')
    os.makedirs(base_dir, exist_ok = True)
    os.makedirs('done', exist_ok = True)
    
    base_dir_filename = os.path.join('done', f'{base_dir}.json')
    
    if os.path.exists(base_dir_filename):
        continue
    
    need_break = False
    last_texts = []
    for i in range(0, 100000, per_batch):
        filename = os.path.join(base_dir, f'{i}.json')
        if os.path.exists(filename):
            continue
        aranged = np.arange(i, i + per_batch, limit)
        urls = []
        for a_ in aranged:
            if a_ == 0:
                a_ = ''
            else:
                a_ = f'/+{a_}'
            url = f'https://forum.lowyat.net/{topic}{a_}'
            urls.append(url)
        
        texts = []
        
        with ThreadPoolExecutor(max_workers=worker_size) as executor:
            futures = {executor.submit(get_text, item): item for item in urls}
            
            for future in as_completed(futures):
                r = future.result()
                texts.extend(r)
                if not len(r):
                    need_break = True
                    
                s = set(texts)
                if len(set(last_texts) & s) == len(s):
                    need_break = True
                    
                last_texts = texts[:]
        
        with open(filename, 'w') as fopen:
            json.dump(texts, fopen)
            
        time.sleep(random.uniform(0.5, 1.5))
        
        if need_break:
            break
    
    with open(base_dir_filename, 'w') as fopen:
        json.dump({'status': True}, fopen) 

In [67]:
!mkdir kopitiam

In [69]:
from glob import glob
files = glob('topic-*/*.json')
len(files)

64664

In [70]:
!mv topic-* kopitiam

In [71]:
!du -hs kopitiam

401M	kopitiam


In [74]:
# !zip -r kopitiam.zip kopitiam

In [80]:
texts = []

files = glob('kopitiam/topic-*/*.json')
for f in tqdm(files):
    with open(f) as fopen:
        texts.extend(json.load(fopen))

100%|███████████████████████████████████| 64664/64664 [00:07<00:00, 9088.12it/s]


In [82]:
texts = list(set(texts))
len(texts)

755196

In [84]:
texts[-10:]

['122 kerusi deposit kena burnt... Who need your service la dey\n\nIt is like saying people in cheras need MCA Mike service though deposit burnt macam siao',
 'Very nice',
 "Ford drilling or kena drill?\n\nDon't understand what is happening.",
 'Can sue parents back now?',
 'how much max you guys collect the lazada bonus? today able to collect once only',
 'https://s.lazada.com.my/s.0QOX5\n\nThanks in advance',
 'bujang one and household one same page?',
 "Cukur my prop didn't get vacant property tax",
 'QUOTE(phas3r @ Jul 25 2021, 11:44 AM)early 2000s my fave was aimar saviola riquelme, they were quite good in CM games\n\nfew days ago saviola got play in barcelona legend games, but ronaldinho was the star of the show\n\n\nSUb0co5AgSc\n\nRiquelme is rarely paired with Aimar, before. \nEither one of them became substitution.\nI feel this is one of the weakness \nshown by their coach at that time.',
 "What happen ? Lazy to watch ...don't want to see gore."]

In [86]:
with open('kopitiam.json', 'w') as fopen:
    json.dump(texts, fopen)